In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta
import plotly.express as px
import plotly.graph_objects as go
import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly as py
import statsmodels
from scipy import stats

# Consumption

In [3]:
index = pd.read_excel('data/5. display_feature.xlsx', 'Index', header = 1)

In [4]:
# comsumption
comsumption = pd.DataFrame()
for i in range(1,66):
    data = pd.read_excel('data/5. display_feature.xlsx', str(i))
    data = data.iloc[7:29]
    data.iloc[0,0] = 'SKU'
    data.iloc[0,] = data.iloc[0,].str[-8:]
    data.columns = data.iloc[0,]
    data = data.iloc[1:, :]
    data.insert(1, 'Region', index[index.Index==i].Geography.item(), True)
    data = pd.melt(data, id_vars = ['SKU', 'Region'], var_name = 'end_of_week', value_name = 'consumption')
    # replace NaNs
    data['consumption'] = data['consumption'].replace(np.nan,0).astype(int)

    # add Platform
    data['Platform'] = data['SKU'].map({'33 KH DINNER 32 PK': 'Dinner roll',
                                     '11 KH DINNER 24 PK': 'Dinner roll',
                                     '38 KH Slider Buns 0073435060801': 'Sandwich',
                                     '07C KH Total Hamburger Buns': 'Sandwich',
                                     '30 KH DINNER 18 PK': 'Dinner roll',
                                     '15 KH Orig Sliced 16oz 7343509330': 'Bread',
                                     '31 KH Orig Dnr 18pk 7343507018': 'Dinner roll',
                                     '22 KH Orig Dnr 24pk 7343500230': 'Dinner roll',
                                     '10 KH DINNER 12 PK': 'Dinner roll',
                                     '20 KH Hny Wht Dnr 12pk 7343500005': 'Dinner roll',
                                     '07D KH Total Hot Dog Buns': 'Sandwich',
                                     '13 KH Orig Rnd Loaf 16oz 7343500001': 'Bread',
                                     '34 KH Orig Dnr 32pk 7343507032': 'Dinner roll',
                                     '12 KH Mini Sub': 'Sandwich',
                                     '16 KH Orig Dnr 4pk 7343500006': 'Dinner roll',
                                     '21 KH Svry Btr Dnr 12pk 7343500201': 'Dinner roll',
                                     '18 KH Svry Btr Dnr 4pk 7343500225': 'Dinner roll',
                                     '19 KH Orig Dnr 12pk 7343500004': 'Dinner roll',
                                     '29 KH Svry Btr Dnr 24pk 7343500233': 'Dinner roll',
                                     '09 KH DINNER 4 PK': 'Dinner roll'}) 
    # date
    data['end_of_week'] = pd.to_datetime(data['end_of_week'])
    data['start_of_week'] = data['end_of_week'] - timedelta(days=6)
    col_name='start_of_week'
    col = data.pop(col_name)
    data.insert(2, col_name, col)

    # drop rows
    for i in ['33 KH DINNER 32 PK','11 KH DINNER 24 PK',
              '10 KH DINNER 12 PK','09 KH DINNER 4 PK',"04 Total King's Hawaiian",'30 KH DINNER 18 PK']:
        data.drop(data[data['SKU'] == i].index, inplace = True)
    data.reset_index(drop = True)   
    comsumption = comsumption.append(data,ignore_index=True)

In [5]:
comsumption.shape

(288684, 6)

# ACV

In [6]:
index = pd.read_excel('data/5.1. 2016-2020 ACV by week_USC_UPDATED CONFIDENTIAL.xlsx', 'Index', header = 1)

In [7]:
# acv
acv = pd.DataFrame()
for i in range(1,66):
    data = pd.read_excel('data/5.1. 2016-2020 ACV by week_USC_UPDATED CONFIDENTIAL.xlsx', str(i))
    data = data.iloc[7:29]
    data.iloc[0,0] = 'SKU'
    data.iloc[0,] = data.iloc[0,].str[-8:]
    data.columns = data.iloc[0,]
    data = data.iloc[1:, :]
    data.insert(1, 'Region', index[index.Index==i].Geography.item(), True)
    data = pd.melt(data, id_vars = ['SKU', 'Region'], var_name = 'end_of_week', value_name = 'acv')
    # replace NaNs
    data['acv'] = data['acv'].replace(np.nan,0).astype(int)

    # add Platform
    data['Platform'] = data['SKU'].map({'33 KH DINNER 32 PK': 'Dinner roll',
                                     '11 KH DINNER 24 PK': 'Dinner roll',
                                     '38 KH Slider Buns 0073435060801': 'Sandwich',
                                     '07C KH Total Hamburger Buns': 'Sandwich',
                                     '30 KH DINNER 18 PK': 'Dinner roll',
                                     '15 KH Orig Sliced 16oz 7343509330': 'Bread',
                                     '31 KH Orig Dnr 18pk 7343507018': 'Dinner roll',
                                     '22 KH Orig Dnr 24pk 7343500230': 'Dinner roll',
                                     '10 KH DINNER 12 PK': 'Dinner roll',
                                     '20 KH Hny Wht Dnr 12pk 7343500005': 'Dinner roll',
                                     '07D KH Total Hot Dog Buns': 'Sandwich',
                                     '13 KH Orig Rnd Loaf 16oz 7343500001': 'Bread',
                                     '34 KH Orig Dnr 32pk 7343507032': 'Dinner roll',
                                     '12 KH Mini Sub': 'Sandwich',
                                     '16 KH Orig Dnr 4pk 7343500006': 'Dinner roll',
                                     '21 KH Svry Btr Dnr 12pk 7343500201': 'Dinner roll',
                                     '18 KH Svry Btr Dnr 4pk 7343500225': 'Dinner roll',
                                     '19 KH Orig Dnr 12pk 7343500004': 'Dinner roll',
                                     '29 KH Svry Btr Dnr 24pk 7343500233': 'Dinner roll',
                                     '09 KH DINNER 4 PK': 'Dinner roll'}) 
    # date
    data['end_of_week'] = pd.to_datetime(data['end_of_week'])
    data['start_of_week'] = data['end_of_week'] - timedelta(days=6)
    col_name='start_of_week'
    col = data.pop(col_name)
    data.insert(2, col_name, col)

    # drop rows
    for i in ['33 KH DINNER 32 PK','11 KH DINNER 24 PK',
              '10 KH DINNER 12 PK','09 KH DINNER 4 PK',"04 Total King's Hawaiian",'30 KH DINNER 18 PK']:
        data.drop(data[data['SKU'] == i].index, inplace = True)
    data.reset_index(drop = True)   
    acv = acv.append(data,ignore_index=True)

In [8]:
acv.shape

(288684, 6)

# Average price

In [9]:
index = pd.read_excel('data/3. 2016-2020 Avg price by SKU by week USC CONFIDENTIAL.xlsx', 'Index', header = 1)

In [ ]:
# average price
average_price = pd.DataFrame()
for i in range(1,66):
    data = pd.read_excel('data/3. 2016-2020 Avg price by SKU by week USC CONFIDENTIAL.xlsx', str(i))
    data = data.iloc[7:29]
    data.iloc[0,0] = 'SKU'
    data.iloc[0,] = data.iloc[0,].str[-8:]
    data.columns = data.iloc[0,]
    data = data.iloc[1:, :]
    data.insert(1, 'Region', index[index.Index==i].Geography.item(), True)
    data = pd.melt(data, id_vars = ['SKU', 'Region'], var_name = 'end_of_week', value_name = 'average_price')
    # replace NaNs
    data['average_price'] = data['average_price'].replace(np.nan,0)

    # add Platform
    data['Platform'] = data['SKU'].map({'33 KH DINNER 32 PK': 'Dinner roll',
                                     '11 KH DINNER 24 PK': 'Dinner roll',
                                     '38 KH Slider Buns 0073435060801': 'Sandwich',
                                     '07C KH Total Hamburger Buns': 'Sandwich',
                                     '30 KH DINNER 18 PK': 'Dinner roll',
                                     '15 KH Orig Sliced 16oz 7343509330': 'Bread',
                                     '31 KH Orig Dnr 18pk 7343507018': 'Dinner roll',
                                     '22 KH Orig Dnr 24pk 7343500230': 'Dinner roll',
                                     '10 KH DINNER 12 PK': 'Dinner roll',
                                     '20 KH Hny Wht Dnr 12pk 7343500005': 'Dinner roll',
                                     '07D KH Total Hot Dog Buns': 'Sandwich',
                                     '13 KH Orig Rnd Loaf 16oz 7343500001': 'Bread',
                                     '34 KH Orig Dnr 32pk 7343507032': 'Dinner roll',
                                     '12 KH Mini Sub': 'Sandwich',
                                     '16 KH Orig Dnr 4pk 7343500006': 'Dinner roll',
                                     '21 KH Svry Btr Dnr 12pk 7343500201': 'Dinner roll',
                                     '18 KH Svry Btr Dnr 4pk 7343500225': 'Dinner roll',
                                     '19 KH Orig Dnr 12pk 7343500004': 'Dinner roll',
                                     '29 KH Svry Btr Dnr 24pk 7343500233': 'Dinner roll',
                                     '09 KH DINNER 4 PK': 'Dinner roll'}) 
    # date
    data['end_of_week'] = pd.to_datetime(data['end_of_week'])
    data['start_of_week'] = data['end_of_week'] - timedelta(days=6)
    col_name='start_of_week'
    col = data.pop(col_name)
    data.insert(2, col_name, col)

    # drop rows
    for i in ['33 KH DINNER 32 PK','11 KH DINNER 24 PK',
              '10 KH DINNER 12 PK','09 KH DINNER 4 PK',"04 Total King's Hawaiian",'30 KH DINNER 18 PK']:
        data.drop(data[data['SKU'] == i].index, inplace = True)
    data.reset_index(drop = True)   
    average_price = average_price.append(data,ignore_index=True)

In [10]:
average_price.shape

(288684, 6)

# Merge

In [11]:
merge_data = average_price.copy()
merge_data['consumption'] = comsumption['consumption']
merge_data['acv'] = acv['acv']
col_name="Platform"
col = merge_data.pop(col_name)
merge_data.insert(1, col_name, col)

In [12]:
merge_data['year'] = merge_data['end_of_week'].dt.year
merge_data = merge_data.loc[merge_data['Region'] != 'Total US - Multi Outlet',].\
            reset_index(drop=True)
merge_data['Region'] = merge_data['Region'].map(lambda x: x.rstrip(' - Multi Outlet'))
merge_data.head()

,SKU,Platform,Region,start_of_week,end_of_week,average_price,consumption,acv,year
0,34 KH Orig Dnr 32pk 7343507032,Dinner roll,"Albany, NY",2015-01-05,2015-01-11,0.000000,0,0,2015
1,22 KH Orig Dnr 24pk 7343500230,Dinner roll,"Albany, NY",2015-01-05,2015-01-11,4.605175,739,21,2015
2,29 KH Svry Btr Dnr 24pk 7343500233,Dinner roll,"Albany, NY",2015-01-05,2015-01-11,4.480000,37,9,2015
3,31 KH Orig Dnr 18pk 7343507018,Dinner roll,"Albany, NY",2015-01-05,2015-01-11,3.480000,75,9,2015
4,19 KH Orig Dnr 12pk 7343500004,Dinner roll,"Albany, NY",2015-01-05,2015-01-11,3.143336,232,32,2015
